In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, classification_report, multilabel_confusion_matrix
import tensorflow as tf
import datetime, os
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Flatten, concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.constraints import MaxNorm
from sklearn.model_selection import GridSearchCV
from splitrepeat import splitrepeat_cv

config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8))
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

features=['moca_digits','moca_letters','moca_serial7','MMSPELL_early','MMSPELL_late','ADAS_Q1','moca_orient','ADAS_Q7','ADAS_Q8','ADAS_Q9','MMDATE',
       'MMYEAR', 'MMMONTH', 'MMDAY', 'MMSEASON', 'MMHOSPIT', 'MMFLOOR','MMCITY', 'MMAREA', 'MMSTATE','nbspan_forward',
       'faq7','faq8','moca_recall','ADAS_Q4','MMBALLDL', 'MMFLAGDL', 'MMTREEDL', 'LDELTOTAL', 'AVRECALL', 'AVDEL30MIN', 'AVDELTOT', 'AVDELERR2',
       'faq9','moca_clock','MMDRAW','CLOCKCIRC', 'CLOCKSYM','CLOCKNUM', 'CLOCKHAND', 'CLOCKTIME', 'COPYCIRC', 'COPYSYM',
       'COPYNUM', 'COPYHAND', 'COPYTIME', 'tmab_time', 'TRAAERRCOM','TRAAERROM', 'TRABERRCOM', 'TRABERROM','moca_naming','moca_repeat','moca_fluency',
       'moca_similarities','ADAS_Q2', 'ADAS_Q5','ADAS_Q10','ADAS_Q11','ADAS_Q12','MMBALL', 'MMFLAG', 'MMTREE',
       'MMWATCH', 'MMPENCIL','MMREPEAT','MMREAD', 'MMWRITE', 'CATANIMSC', 'CATVEGESC', 'moca_visuo_exec','ADAS_Q3','ADAS_Q6','ADAS_Q13','MMHAND', 'MMFOLD',
       'MMONFLR','nbspan_backward','faq1','faq2','faq3', 'faq4','faq5','faq6','faq10','PXGENAPP', 'PXHEADEY', 'PXNECK', 'PXCHEST',
       'PXHEART', 'PXABDOM', 'PXEXTREM', 'PXPERIPH', 'PXSKIN', 'PXMUSCUL', 'e_memory_pt', 'e_lang_pt', 'e_visspat_pt', 'e_plan_pt', 'e_organ_pt',
       'e_divatt_pt','e_memory_cg', 'e_lang_cg', 'e_visspat_cg','e_plan_cg', 'e_organ_cg', 'e_divatt_cg']


categorical_features = np.in1d(features, ['MMONFLR', 'faq10', 'faq6', 'COPYSYM', 'COPYNUM', 'faq5', 'MMTREE', 'COPYHAND', 'moca_clock', 'MMSEASON', 'moca_letters', 
                                          'MMBALL', 'faq9', 'MMFLOOR', 'MMDRAW', 'MMMONTH', 'PXGENAPP', 'MMWATCH', 'CLOCKCIRC', 'faq8', 'MMHOSPIT', 'moca_naming', 
                                          'PXEXTREM', 'CLOCKNUM', 'PXMUSCUL', 'faq1', 'MMTREEDL', 'CLOCKTIME', 'PXABDOM', 'MMFLAG', 'COPYCIRC', 'MMAREA', 'faq3', 
                                          'moca_digits', 'CLOCKHAND', 'MMREAD', 'MMYEAR', 'MMREPEAT', 'moca_visuo_exec','MMHAND', 'MMBALLDL', 'PXCHEST', 'MMDATE', 
                                          'MMFLAGDL',  'MMSTATE','moca_repeat', 'MMFOLD', 'MMPENCIL', 'MMDAY',  'faq2', 'PXHEART', 'CLOCKSYM', 'faq4', 'moca_serial7', 
                                          'faq7', 'MMCITY', 'PXHEADEY', 'COPYTIME', 'PXPERIPH', 'PXSKIN', 'moca_fluency', 'moca_similarities', 'PXNECK', 'MMWRITE',
                                          'e_memory_pt', 'e_lang_pt', 'e_visspat_pt', 'e_plan_pt', 'e_organ_pt', 'e_divatt_pt','e_memory_cg', 'e_lang_cg', 
                                          'e_visspat_cg','e_plan_cg', 'e_organ_cg', 'e_divatt_cg'])

data = pd.read_csv('../data/interim/data_adni.csv')
X = data[features]
y = data['CDGLOBAL']
y.replace({2:0,1:1},inplace=True)

In [10]:
# Divide data in half for tuning, to reduce leakage when using split/repeat cross-validation
X, X_test, y, y_test = train_test_split(X, y, train_size=0.25, random_state=33433, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=33433, stratify=y)     

In [11]:
y.value_counts()

0.0    399
1.0    383
Name: CDGLOBAL, dtype: int64

# RF Classifier

In [12]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_features=.5, random_state=0)

%time \
df = splitrepeat_cv(X,y,rf,splits=[111,222,333,444],repeats=[111,222,333,444,555])
df.describe()

Wall time: 5.45 s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.794000,0.901042,0.846429,0.847521,0.901042,0.794000,0.846429,0.847521,0.901042,0.794000,0.846429,0.847521
std,0.035004,0.028577,0.011693,0.011497,0.028577,0.035004,0.011693,0.011497,0.028577,0.035004,0.011693,0.011497
min,0.720000,0.854167,0.821429,0.823542,0.854167,0.720000,0.821429,0.823542,0.854167,0.720000,0.821429,0.823542
25%,0.765000,0.885417,0.836735,0.838542,0.885417,0.765000,0.836735,0.838542,0.885417,0.765000,0.836735,0.838542
50%,0.810000,0.901042,0.846939,0.848333,0.901042,0.810000,0.846939,0.848333,0.901042,0.810000,0.846939,0.848333
75%,0.812500,0.927083,0.853316,0.854219,0.927083,0.812500,0.853316,0.854219,0.927083,0.812500,0.853316,0.854219
max,0.840000,0.947917,0.867347,0.868750,0.947917,0.840000,0.867347,0.868750,0.947917,0.840000,0.867347,0.868750


## Random Search

In [13]:
from sklearn.model_selection import RandomizedSearchCV

# Search parameters
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['sqrt', 'log2', .3, .5, .7, .9]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

search_params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [14]:
rf_search_rand = RandomizedSearchCV(rf, search_params, n_iter=400,cv=3,verbose=2,random_state=33433, n_jobs=-1)
rf_search_rand.fit(X_train, y_train)

Fitting 3 folds for each of 400 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:  9.0min finished


RandomizedSearchCV(cv=3,
                   estimator=RandomForestClassifier(max_features=0.5,
                                                    random_state=555),
                   n_iter=400, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['sqrt', 'log2', 0.3,
                                                         0.5, 0.7, 0.9],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                        

In [15]:
rf_search_rand.best_params_

{'n_estimators': 400,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 'log2',
 'max_depth': 30,
 'bootstrap': False}

In [16]:
best = rf_search_rand.best_estimator_
df_rand = splitrepeat_cv(X,y,best,splits=[10,20,30],repeats=[10,20,30])
df_rand.describe()

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.821111,0.902778,0.861111,0.861944,0.902778,0.821111,0.861111,0.861944,0.902778,0.821111,0.861111,0.861944
std,0.066228,0.030370,0.021462,0.020599,0.030370,0.066228,0.021462,0.020599,0.030370,0.066228,0.021462,0.020599
min,0.730000,0.854167,0.831633,0.833542,0.854167,0.730000,0.831633,0.833542,0.854167,0.730000,0.831633,0.833542
25%,0.740000,0.885417,0.836735,0.838958,0.885417,0.740000,0.836735,0.838958,0.885417,0.740000,0.836735,0.838958
50%,0.860000,0.895833,0.867347,0.867500,0.895833,0.860000,0.867347,0.867500,0.895833,0.860000,0.867347,0.867500
75%,0.860000,0.927083,0.877551,0.877917,0.927083,0.860000,0.877551,0.877917,0.927083,0.860000,0.877551,0.877917
max,0.880000,0.947917,0.882653,0.883125,0.947917,0.880000,0.882653,0.883125,0.947917,0.880000,0.882653,0.883125


## Grid Search

In [17]:
from sklearn.model_selection import GridSearchCV

search_params = {'n_estimators': [300, 400, 500],
               'max_features': [.2, 'sqrt','log2'],
               'max_depth': [20, 30, 40],
               'min_samples_split': [9, 10, 11],
               'min_samples_leaf': [1, 2],
               'bootstrap': [False, True]}

rf_search_grid = GridSearchCV(rf, search_params, 
                          cv = 3, n_jobs = -1, verbose = 2)
rf_search_grid.fit(X_train, y_train)

Fitting 3 folds for each of 324 candidates, totalling 972 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   18.7s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   41.8s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 972 out of 972 | elapsed:  2.0min finished


GridSearchCV(cv=3,
             estimator=RandomForestClassifier(max_features=0.5,
                                              random_state=555),
             n_jobs=-1,
             param_grid={'bootstrap': [False, True], 'max_depth': [20, 30, 40],
                         'max_features': [0.2, 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [9, 10, 11],
                         'n_estimators': [300, 400, 500]},
             verbose=2)

In [18]:
rf_search_grid.best_params_

{'bootstrap': False,
 'max_depth': 30,
 'max_features': 'log2',
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 400}

In [19]:
best = rf_search_grid.best_estimator_
df_grid = splitrepeat_cv(X,y,best,splits=[10,20,30],repeats=[10,20,30])
df_grid.describe()

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.821111,0.902778,0.861111,0.861944,0.902778,0.821111,0.861111,0.861944,0.902778,0.821111,0.861111,0.861944
std,0.066228,0.030370,0.021462,0.020599,0.030370,0.066228,0.021462,0.020599,0.030370,0.066228,0.021462,0.020599
min,0.730000,0.854167,0.831633,0.833542,0.854167,0.730000,0.831633,0.833542,0.854167,0.730000,0.831633,0.833542
25%,0.740000,0.885417,0.836735,0.838958,0.885417,0.740000,0.836735,0.838958,0.885417,0.740000,0.836735,0.838958
50%,0.860000,0.895833,0.867347,0.867500,0.895833,0.860000,0.867347,0.867500,0.895833,0.860000,0.867347,0.867500
75%,0.860000,0.927083,0.877551,0.877917,0.927083,0.860000,0.877551,0.877917,0.927083,0.860000,0.877551,0.877917
max,0.880000,0.947917,0.882653,0.883125,0.947917,0.880000,0.882653,0.883125,0.947917,0.880000,0.882653,0.883125


In [20]:
rf = RandomForestClassifier(n_estimators=400, max_features='log2', max_depth = 30, min_samples_split = 10, min_samples_leaf = 1, bootstrap=False, random_state=33433)

%time \
df = splitrepeat_cv(X,y,rf,splits=[111,222,333,444],repeats=[111,222,333,444,555])
df.describe()

Wall time: 10.5 s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.795500,0.871875,0.832908,0.833688,0.871875,0.795500,0.832908,0.833688,0.871875,0.795500,0.832908,0.833688
std,0.025438,0.035138,0.026009,0.026122,0.035138,0.025438,0.026009,0.026122,0.035138,0.025438,0.026009,0.026122
min,0.760000,0.822917,0.806122,0.806458,0.822917,0.760000,0.806122,0.806458,0.822917,0.760000,0.806122,0.806458
25%,0.777500,0.843750,0.811224,0.812188,0.843750,0.777500,0.811224,0.812188,0.843750,0.777500,0.811224,0.812188
50%,0.795000,0.864583,0.821429,0.822708,0.864583,0.795000,0.821429,0.822708,0.864583,0.795000,0.821429,0.822708
75%,0.820000,0.898438,0.843112,0.843750,0.898438,0.820000,0.843112,0.843750,0.898438,0.820000,0.843112,0.843750
max,0.840000,0.937500,0.882653,0.883750,0.937500,0.840000,0.882653,0.883750,0.937500,0.840000,0.882653,0.883750
